## GLANCE: Global Actions In A Nutshell for Counterfactual Explainability

**GLANCE** is a versatile and adaptive framework for generating *global counterfactual explanations*. <br>
These explanations are expressed as actions that offer recourse to large population subgroups.<br> The framework aims to provide explanations and insights, ensuring that the actions benefit as many individuals as possible.

## Preliminaries

### Import Dependencies 
As usual in python, the first step is to import all necessary packages.



In [1]:
from xgboost import XGBClassifier
import pandas as pd
from glance.glance.glance import GLANCE
from utils import load_models, preprocess_datasets

## Load Data and Model to be used for explanations
This will serve as the demonstrative model, which we will then treat as a black box and apply our algorithm.
Of course, any model can be used in its place.



In [2]:
dataset = "german_credit"
model_name = "xgb"

train_dataset, data, X_train, y_train, X_test, y_test, affected, _unaffected, model, feat_to_vary, target_name = (
    preprocess_datasets(dataset, load_models(dataset, model_name), model_name)
)

[15:21:33] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.

Accuracy: 0.74


## GLANCE 
GLANCE is a clustering-based algorithm designed to generate global counterfactual explanations. <br>It starts by forming initial clusters and gradually merges them until the number of clusters matches the user-defined final_clusters parameter.<br> From each of these final clusters, the best action is selected, and together, these actions form the global explanation.

GLANCE framework is loaded with:
 - the model to be explained
 - number of initial clusters, 
 - number of final clusters, from each of which the best action is extracted
 - number of local counterfactuals, that the Local Counterfactual Method generates for each centroid of the initial clusters

 GLANCE algorithm allows the users to specify the number of global actions generated and serves as a tool to explain and debug ML models.

In [3]:
global_method = GLANCE(
    model, initial_clusters=10, final_clusters=3, num_local_counterfactuals=5
)
global_method.fit(
    data.drop(columns=["Status"]), data["Status"], train_dataset, feat_to_vary
)

In [4]:
clusters, clusters_res = global_method.explain_group(affected)

100%|██████████| 20/20 [00:00<00:00, 289.06it/s]

Action 1 
Existing-Account-Status = A14 
Num-People +0.3999999999999999 

Effectiveness: 100.00%	Cost: 1.00


Action 2 
Savings-Account = A65 
Telephone = A192 

Effectiveness: 100.00%	Cost: 1.50


Action 3 
Existing-Account-Status = A14 
Residence +0.5 

Effectiveness: 100.00%	Cost: 1.00


TOTAL EFFECTIVENESS: 100.00%
TOTAL COST: 1.00


## GLANCE Output
GLANCE generates a set of final actions, with a focus on their overall impact when applied to the entire affected population. While each action is initially associated with a specific cluster, the key metrics we prioritize are the *Total Effectiveness* and *Total Cost* across the whole population.

- *Total Effectiveness* is the percentage of individuals that achieve the favorable outcome, if each one of the final actions is applied to the whole affected population.<br>
- *Total Cost* is calculated as the mean recourse cost of the whole set of final actions over the entire population.

Additionally, for each generated action the suggested changes are also reported, as well as the *effectiveness* and *cost* they achieve on the population of the cluster they were extracted from. More specifically:

- *Effectiveness*, for each cluster-action pair ($C$, $a$), represents the percentage of individuals in $C$ who get the favorable outcome when the action $a$ is applied.
<br>
- *Cost*, for each cluster-action pair ($C$, $a$), is the mean recourse cost computed when the action $a$ is applied to the individuals of cluster $C$.
<br>

## GLANCE Modularity
Our framework is highly **modular**, allowing users to customize various aspects of it. <br>

Specifically:
- **Choice of Local Counterfactual Methods**: Users can select from a variety of local counterfactual methods to generate candidate counterfactual explanations, such as:
    - **NearestNeighbors**: When queried to provide *k* counterfactuals for an affected individual, it retrieves the k nearest neighbors from the set of unaffected instances based on their proximity to the affected individual.
    - **Random Sampling**: To find counterfactuals for an affected instance, this method iteratively modifies its features one at a time. The process begins by randomly altering one feature at a time, generating multiple new candidate instances

- **Strategy for Selecting Actions**: Additionally, users can choose different strategies for selecting the best actions from the generated counterfactuals. This enables fine-tuning of the process, allowing for the optimal balance between effectiveness and recourse cost, based on user-defined preferences.
    - **max-eff** : Selects actions based on maximizing the effectiveness.
    - **low-cost** : Selects the action with the lowest cost that flips a sufficient number of instances.
    - **mean-act** : Selects the mean action from a set of candidate actions.

In order to use them, the user should provide the **fit** method with the **cf_generator** and **cluster_action_choice_algo** variables and choose the methods of his/hers liking.

In [6]:
global_method = GLANCE(
    model, initial_clusters=100, final_clusters=3, num_local_counterfactuals=10
)
global_method.fit(
    data.drop(columns=["Status"]),
    data["Status"],
    train_dataset,
    feat_to_vary,
    cf_generator="NearestNeighbors",
)

In [7]:
clusters, clusters_res = global_method.explain_group(affected)

100%|██████████| 190/190 [00:00<00:00, 201.39it/s]

Action 1 
Existing-Account-Status = A14 
Month-Duration -3.0 
Credit-History = A32 
Purpose = A43 
Credit-Amount -1.0 
Savings-Account = A65 
Present-Employment = A75 
Residence +1.0 
Age +10.0 
Housing = A152 

Effectiveness: 100.00%	Cost: 10.45


Action 2 
Credit-History = A34 
Purpose = A41 
Credit-Amount +80.0 
Savings-Account = A65 
Present-Employment = A73 
Sex = A93 
Age +4.0 
Existing-Credits -1.0 

Effectiveness: 100.00%	Cost: 8.39


Action 3 
Existing-Account-Status = A14 
Credit-History = A34 
Purpose = A46 
Credit-Amount +4.0 
Present-Employment = A73 
Age -14.0 
Installment = A143 
Telephone = A191 

Effectiveness: 100.00%	Cost: 6.45


TOTAL EFFECTIVENESS: 100.00%
TOTAL COST: 6.66
